## Here we try to sample using Black Jax which include HMC/NUTS/Tempered Hamiltonian

In [ ]:
# !pip install numpy
# %pip install jax==0.4.28
# !pip install blackjax
# !pip install estival
# !pip install PyYaml
# !pip install pandas
# !pip install nbformat --upgrade

# !pip install jaxlib==0.4.28
# !pip install pymc
# !pip install --upgrade --force-reinstall mingw
# %pip --upgrade m2w64-toolchain
# %pip install multiprocess

In [ ]:
%pip install summerepi2==1.3.6


In [ ]:
import numpy as np
import jax.numpy as jnp
import blackjax
#import blackjax.smc.resampling as resampling
import pandas as pd
import jax
import pymc as pm
import time as time

from datetime import date
import arviz as az
import matplotlib.pyplot as plt
from collections import namedtuple



In [ ]:
# Combining tagets and prior with our summer2 model in a BayesianCompartmentalModel (BCM)
from estival.model import BayesianCompartmentalModel

from calibs_utilities import get_all_priors, get_targets, load_data

from models.models import model1 #All the models we design for the test


## Importing our taget and prior data

In [ ]:
pd.options.plotting.backend = "plotly"
output_labels = {"index": "time", "value": "number infectious"}

targets_yml = './data/target_yml.yml'
targets = load_data(targets_yml)
targets_data = targets['active_cases']



#Names of parameters and their ranges for the priors definitions
params = {
    "contact_rate": (0.0,2.0),
    "recovery_rate": (0.0,1.0)

}

targets = get_targets(targets_yml)
priors = get_all_priors(params)

targets_data.plot(kind="scatter",labels=output_labels)

## Building a BCM model with estival

In [ ]:
model_1 = model1()
default_params = {
    "contact_rate": 0.5,
    "recovery_rate": 0.0,
    #"active_cases_dispersion": 0.5,
}
bcm = BayesianCompartmentalModel(model_1, default_params,priors, targets)



### LogLikelihood definition from the bcm model

In [ ]:
params_tuple = namedtuple("model_params", ["contact_rate", "recovery_rate"]) #,typename=np.float64)


#Defining a callable Loglikelihood from the bcm model
def Loglikehood(params_tuple):
        params = {'contact_rate': params_tuple.contact_rate, 'recovery_rate': params_tuple.recovery_rate}  #, 'cdr': 0.2}
        val = bcm.loglikelihood(**params)
        return val.astype(np.float64)

### Initial parameters

In [ ]:
## Initial parameters for the chains
# For now we initialise with default chosen constant values for all chains
# We can later experiment a Uniform initialisation
import tensorflow_probability.substrates.jax as tfp

tfd = tfp.distributions

def init_param_fn(seed): #Permet d'initialiser les parametres pour chaque chaine 
    """
    initialize recovery_rate & contact_rate for the chains 
    """
    key1, key2 = jax.random.split(seed, 2)
    return params_tuple(
    #    contact_rate = jnp.ones(n_chains)*0.15,
    #     recovery_rate = jnp.ones(n_chains)*0.05,
    contact_rate = tfd.Uniform(0, 1).sample(seed=key1).astype(np.float64),
    recovery_rate = tfd.Uniform(0, 0.5).sample(seed=key2).astype(np.float64),
    )


In [ ]:
# we use n_chains chains for sampling
rng_key = jax.random.key(int(date.today().strftime("%Y%m%d")))
n_chains = 2

rng_key, init_key, warmup_key = jax.random.split(rng_key, 3)
init_keys = jax.random.split(init_key, n_chains)
init_params = jax.vmap(init_param_fn)(init_keys)


#Checking result with jax vmap to map the function to multiple args
jax.vmap(Loglikehood)(init_params)



### Now we use blackjax’s window adaption algorithm to get NUTS kernel and initial states. Window adaption algorithm will automatically configure inverse_mass_matrix and step size

In [ ]:
%%time
#rng_key = jax.random.key(int(date.today().strftime("%Y%m%d")))
rng_key, warmup_key = jax.random.split(rng_key, 2)

hmc_parameters = dict(num_integration_steps = 10)

warmup = blackjax.window_adaptation(blackjax.hmc, logdensity_fn = Loglikehood,target_acceptance_rate = 0.6,**hmc_parameters)

# we use n_chains chains for sampling
@jax.vmap
def call_warmup(seed, param):
    (initial_states, tuned_params), _ = warmup.run(seed, param, 1000)
    return initial_states, tuned_params


warmup_keys = jax.random.split(warmup_key, n_chains)

initial_states, tuned_params = jax.jit(call_warmup)(warmup_keys, init_params)

## Sampling wiht HMC from BlackJax

#### An inference loop for multiple chains sampling

In [ ]:
def inference_loop_multiple_chains(
    rng_key, initial_states, tuned_params, log_prob_fn, num_samples, num_chains
):
    kernel = blackjax.hmc.build_kernel()
    @jax.jit  #Comment to assess the performance
    def step_fn(key, state, **params):
        return kernel(key, state, log_prob_fn, **params)

    def one_step(states, rng_key):
        keys = jax.random.split(rng_key, num_chains)
        states, infos = jax.vmap(step_fn)(keys, states, **tuned_params)
        return states, (states, infos)

    keys = jax.random.split(rng_key, num_samples)
    _, (states, infos) = jax.lax.scan(one_step, initial_states, keys)

    return (states, infos)


#### Sampling process

Slow process. Has to be run in parallel

In [ ]:
%%time
n_samples = 5000
rng_key, sample_key = jax.random.split(rng_key)

states, infos = inference_loop_multiple_chains(
        rng_key = sample_key,
        initial_states =  initial_states,
        tuned_params = tuned_params,
        log_prob_fn = Loglikehood,
        num_samples = n_samples,
        num_chains = n_chains,
)

### Derived the trace from the outputs of our sampler

In [ ]:
def arviz_trace_from_states(states, info, burn_in=0):
    position = states.position
    if isinstance(position, jax.Array):  # if states.position is array of samples
        position = dict(samples=position)
    else:
        try:
            position = position._asdict()
        except AttributeError:
            pass

    samples = {}
    for param in position.keys():
        ndims = len(position[param].shape)
        if ndims >= 2:
            samples[param] = jnp.swapaxes(position[param], 0, 1)[
                :, burn_in:
            ]  # swap n_samples and n_chains
            divergence = jnp.swapaxes(info.is_divergent[burn_in:], 0, 1)

        if ndims == 1:
            divergence = info.is_divergent
            samples[param] = position[param]

    trace_posterior = az.convert_to_inference_data(samples)
    trace_sample_stats = az.convert_to_inference_data(
        {"diverging": divergence}, group="sample_stats"
    )
    trace = az.concat(trace_posterior, trace_sample_stats)
    return trace

In [ ]:
# make arviz trace from states
trace = arviz_trace_from_states(states, infos, burn_in=0)
summ_df = az.summary(trace)
summ_df

In [ ]:
trace

In [ ]:
az.plot_trace(trace)
plt.tight_layout()

In [ ]:

import multiprocess as mp
import platform

if platform.system() != "Windows":
    
    mp.set_start_method('forkserver')


from estival.wrappers import pymc as epm
from pymc.sampling import jax as jpm

In [ ]:
with pm.Model() as model:
    variables = epm.use_model(bcm, include_ll=True)
    # The log-posterior value can also be output, but may incur additional overhead
    # Use jacobian=False to get the unwarped value (ie just the 'native' density of the priors
    # without transformation correction factors)
    variables = pm.Deterministic("logp", model.logp(jacobian=False))
    
    # Now call a sampler using the variables from use_model
    idata_HCM = jpm._sample_blackjax_nuts(model,
                                        draws=100,
                                        tune = 100,
                                        chains=4, 
                                        chain_method='parallel',
                                        target_accept=0.5,
                                        progressbar=False,
                                        random_seed=1,
                                        initial_points={"contact_rate": 0.9, "recovery_rate": 0.4},
                                        nuts_kwargs = None)

In [ ]:
from pymc.sampling import jax

jax._sample_blackjax_nuts